# *Datasets*

Os *datasets* (conjuntos de dados) são o principal insumo dos processos de análise de dados e de ciência de dados. Eles são representados normalmente por dados tabulares em formato de planilha onde as linhas são os registros dos acontecimentos e as colunas são as características desses acontecimentos.

Existem variadas formas de salvar um *dataset*, na área de processamento de dados as formas mais comuns envolvem salvar os dados em arquivos nos formatos `.xlsx`, `.csv` ou `.tsv`. Embora também seja possível trabalhar com dados guardados em bases de dados, obtendo as informações por meio de consultas a base.

Neste `notebook` será apresentado o carregamento do *dataset*  [*DETECTING INDIVIDUAL AND COMBINED FINGERS MOVEMENTS*](https://www.rami-khushaba.com/electromyogram-emg-repository.html).

## Detalhes do *dataset*
>É uma boa prática prover um resumo do *dataset* utilizado, pontuando características importantes para o processo da analise dos dados, explicando como ele está estruturado e colocando as devidas refências quando aplicável.

*dataset*: DETECTING INDIVIDUAL AND COMBINED FINGERS MOVEMENTS \[1\]


Dez voluntários, com idade entre 20 e 35 anos foram recrutados para realizarem movimentos com os dedos.
Os voluntários possuem os membros normais e sem desordem neurológica ou muscular. Os voluntários estavam sentados em uma poltrona com o braço apoiado e fixado em uma posição, para evitar efeitos de posições diferentes do membro. Os dados foram coletados utilizando **2 canais EMG** e processados pelo software Bagnoli Desktop EMG Systems da Delsys Inc. Os sinals EMG coletados dos eletrodos foram amplificados utilizando o amplificador Delsys Bagnoli-8 para um ganho total de 1000. Um conversor analógico digital de 12 bits foi utilizado para amostrar o sinal a **4000 Hz**.

Os sinais EMG foram filtrados por um filtro passa-banda entre 20 e 450 Hz com um filtro *notch* para remover as faixas de 50 Hz provenientes da rede elétrica.
Sinais de dez classes de movimentos de dedos, individuais e combinados, foram coletados. Para cada classe foram realizados seis ensaios.

- Flexão individual:
  - Polegar (T-T)
  - Indicador (I-I)
  - Médio (M-M)
  - Anular (R-R)
  - Minimo (L-L)

- Flexão combinada:
  - Polegar-Indicador (T-I)
  - Polegar-Médio (T-M)
  - Polegar-Anular (T-R)
  - Polegar-Minimo (T-L)
  - Punho fechado (HC)

Os arquivos do *dataset* estão organizados da seguinte forma: há uma pasta para cada voluntário, o padrão de nomenclatura das pastas é `EMG-S<número_do_voluntário>`, por exemplo, para o voluntário 1 temos a pasta
 `EMG-S1`, para o voluntário 2 temos a pasta `EMG-S2`, e assim sucessivamente.

Dentro de cada pasta há um arquivo `.csv` para cada coleta. São seis ensaios por classe, totalizando 60 arquivos `.csv`. Cada arquivo possui duas colunas, sem cabeçalho. Cada coluna representa os dados de um canal EMG. A nomenclatura dos arquivos segue o padrão `<nome_da_classe><número_da_coleta>.csv`, por exemplo, a primeira coleta do arquivo da classe Polegar, representado por T-T, tem o nome `T-T1.csv`, o arquivo da segunda coleta tem o nome `T-T2.csv`, e assim sucessivamente.

\[1\] R. N. Khushaba, M. Takruri, S. Kodagoda, and G. Dissanayake, "Toward Improved Control of Prosthetic Fingers Using Surface Electromyogram (EMG) Signals", Expert Systems with Applications, vol 39, no. 12, pp. 10731–10738, 2012.
Disponível em: (https://www.rami-khushaba.com/electromyogram-emg-repository.html)


>Com a análise dos detalhes do *dataset* utilizado descobrimos dados importantes, como a frêquencia de amostragem e o fato de que esse *dataset* já teve seus dados filtratos para remoção de interferências da rede elétrica. Estas informações serão úteis em etapas posteriores.

## Carregando o *dataset*

O carregamento do *dataset* varia conforme o formato que os dados estão armazenados e estruturados. Vimos que temos os dados no formato `.csv`, utilizaremos a biblioteca [pandas](https://pandas.pydata.org) para realizar o carregamento e a biblioteca [numpy](https://numpy.org) para organizar os dados em arrays para facilitar a manipulação destes dados futuramente.

In [1]:
import pandas as pd

# carrega um arquivo .csv para um pandas dataframe
t_t1 = pd.read_csv('./datasets/khushaba/EMG-S2/T-T1.csv', delimiter=',', header=None)

In [3]:
# converte o dataframe do pandas para um numpy array
t_t1 = t_t1.to_numpy()

# imprime o shape do numpy array
print(t_t1.shape)
# neste exemplo devemos ter um shape de (20000, 2), 20000 linhas e 2 colunas, onde cada coluna é um canal.

(20000, 2)


Realizamos o carregamento de um ensaio de uma classe de um voluntário e o tranformamos em um numpy array. É preciso carregar todas os ensaios de cada classe, podemos usufluir do padrão da nomenclaruta dos arquivos e criar um laço de repetição para o carregamento destes dados

In [5]:
import numpy as np

classes = ['T-I','T-L','T-M','T-R','T-T','HC-','I-I','L-L','M-M','R-R']

# variável para armazenar os dados das classes
data = []

for classe in classes:
    # variável para armazear os dados dos ensaios
    trials = []
    for i in range(1, 7): # de 1 a 6 (Qt. de ensaios)
        # carrega o arquivo .csv para um pandas dataframe
        dataframe = pd.read_csv(f'./datasets/khushaba/EMG-S1/{classe}{i}.csv', delimiter=',', header=None)
        
        # converte os dados do um ensaio para numpy array e o adiciona na lista de ensaios
        trials.append(dataframe.to_numpy())
    
    # adiciona os ensaios de uma classe a lista de dados das classes
    data.append(trials)

# transforma os dados das classes em um numpy array
data = np.array(data)

# imprime o shape do numpy array
print(f'{data.shape} - (classes, ensaios, linhas, canais)')
# neste momento devemos ter um shape de (10, 6, 20000, 2), sendo 10 classes, 6 ensaios, 20000 linhas e 2 canais

(10, 6, 20000, 2) - (classes, ensaios, linhas, canais)


O shape atual dos dados não é adequado para os processamentos posterior que serão aplicados sobre eles, o ideal é que as linhas com os dados dos eletrodos estejam no último eixo do numpy array, para corrigir isso podemos utilizar a função *swapaxes* do próprio numpy array

In [8]:
# troca os eixos 2 e 3 do numpy array
data = data.swapaxes(2, 3)

# imprime o shape do numpy array
print(f'{data.shape} - (classes, ensaios, canais, linhas)')
# neste momento devemos ter um shape de (10, 6, 2, 2000), sendo 10 classes, 6 ensaios, 2 canais e 20000 linhas

(10, 6, 2, 20000) - (classes, ensaios, canais, linhas)


**Desafio:** Utilizando o padrão de nomenclatura das pastas que guardam os arquivos do *dataset*, implemente uma função que realizará o carregamento dos dados para todas os voluntários do *dataset*. Essa função deve retornar um dicionário python cuja as chaves sejam os voluntários e o valor de cada chave seja um `numpy array` representando os dados daquele voluntário.

In [9]:
# Sua função aqui
import os
import numpy as np
import pandas as pd

def load_participantes():
    arquivos = os.listdir('./datasets/khushaba')
    participante = [f for f in arquivos if f.startswith("EMG-")]
    classes = ['T-I','T-L','T-M','T-R','T-T','HC-','I-I','L-L','M-M','R-R']

    # variável para armazenar os dados das classes
    data = {}

    for p in participante:
        for classe in classes:
            # variável para armazear os dados dos ensaios
            trials = []
            for i in range(1, 7): # de 1 a 6 (Qt. de ensaios)
                # carrega o arquivo .csv para um pandas dataframe
                dataframe = pd.read_csv(f'./datasets/khushaba/{p}/{classe}{i}.csv', delimiter=',', header=None)
                
                # converte os dados do um ensaio para numpy array e o adiciona na lista de ensaios
                trials.append(dataframe.to_numpy())
            
            # adiciona os ensaios de uma classe a lista de dados das classes
            data[p] = np.array(trials)

        # transforma os dados das classes em um numpy array
        # data = np.array(data)
    return data



In [10]:
people = load_participantes()
people

{'EMG-S8': array([[[-3.084720e-04,  1.026876e-04],
         [-2.611490e-04,  1.002567e-04],
         [-1.885437e-04,  1.026876e-04],
         ...,
         [ 4.232529e-04, -1.007043e-04],
         [ 5.245437e-04, -2.216050e-04],
         [ 6.281036e-04, -3.319715e-04]],
 
        [[ 7.780141e-04,  3.804112e-03],
         [ 7.068673e-04,  3.833771e-03],
         [ 6.306966e-04,  3.710924e-03],
         ...,
         [-1.022208e-03, -2.724935e-04],
         [-1.042304e-03, -3.266234e-04],
         [-9.526815e-04, -3.402368e-04]],
 
        [[ 1.355172e-05, -2.300324e-04],
         [-4.267797e-04, -4.378001e-04],
         [-8.308084e-04, -6.774946e-04],
         ...,
         [-1.668270e-04, -1.206314e-03],
         [-2.169051e-04, -1.047814e-03],
         [-2.559629e-04, -7.607962e-04]],
 
        [[-5.571990e-06, -7.557722e-04],
         [ 3.563199e-04, -6.197994e-04],
         [ 7.374978e-04, -3.760532e-04],
         ...,
         [ 5.179913e-05,  7.124698e-05],
         [ 6.606088e-05

### Melhorando o carregamento dos dados

Durante o desenvolvimento de uma analise de dados é comum precisar rodar o código várias vezes, sendo ncessário carregar sempre os dados dos arquivos do *dataset* e realizar a conversão para numpy array.
É possível, entretando, salvar os dados do numpy array em um arquivo `.npy`, desta forma sempre que o código for executado podemos verificar se o arquivo já existe e realizar um rápido carregamento dos dados já no formato numpy array, sem a necessidade de carregar os arquivos do *dataset* e realizar conversões.

In [2]:
# Salva o numpy array 'data' em './lib/data/converted/s1.npy'
np.save('./datasets/khushaba/s2', data)
# print(np.load('./datasets/khushaba/s2.npy'))
# Lembre-se de que o diretório deve existir, caso contrário uma exceção será lançada

NameError: name 'np' is not defined

**Desafio:** Implemente uma função que carregue os dados de todos os voluntários do *dataset*. Para cada voluntário a função deve verificar a existencia de um arquivo `.npy`, caso o arquivo exista ele deve ser carregado, caso contrário um carregamento dos arquivos do *dataset* deve ser realizado, salvando os dados em uma arquivo `.npy` ao final. Essa função deve retornar um dicionário python contendo os dados de todos os voluntários.

In [14]:
# Sua função aqui
def load_participantes2():
    arquivos = os.listdir('./datasets/khushaba')
    participante = [f for f in arquivos if f.startswith("EMG-")]
    classes = ['T-I','T-L','T-M','T-R','T-T','HC-','I-I','L-L','M-M','R-R']

    # variável para armazenar os dados das classes
    data = {}

    for p in participante:
        arq_numpy = [f for f in p if f.endswith(".npy")]
        if arq_numpy:
            data[p] = np.load(arq_numpy[0])
        else:
            for classe in classes:
                # variável para armazear os dados dos ensaios
                trials = []
                for i in range(1, 7): # de 1 a 6 (Qt. de ensaios)
                    # carrega o arquivo .csv para um pandas dataframe
                    dataframe = pd.read_csv(f'./datasets/khushaba/{p}/{classe}{i}.csv', delimiter=',', header=None)
                    
                    # converte os dados do um ensaio para numpy array e o adiciona na lista de ensaios
                    trials.append(dataframe.to_numpy())
                
                # adiciona os ensaios de uma classe a lista de dados das classes
                data[p] = np.array(trials)
                separa = p.split('-')
                nome = separa[1]
                np.save(f'./datasets/khushaba/{p}/{nome}.npy',data[p])

            # transforma os dados das classes em um numpy array
            # data = np.array(data)
    return data

In [15]:
p2 = load_participantes2()
p2

{'EMG-S8': array([[[-3.084720e-04,  1.026876e-04],
         [-2.611490e-04,  1.002567e-04],
         [-1.885437e-04,  1.026876e-04],
         ...,
         [ 4.232529e-04, -1.007043e-04],
         [ 5.245437e-04, -2.216050e-04],
         [ 6.281036e-04, -3.319715e-04]],
 
        [[ 7.780141e-04,  3.804112e-03],
         [ 7.068673e-04,  3.833771e-03],
         [ 6.306966e-04,  3.710924e-03],
         ...,
         [-1.022208e-03, -2.724935e-04],
         [-1.042304e-03, -3.266234e-04],
         [-9.526815e-04, -3.402368e-04]],
 
        [[ 1.355172e-05, -2.300324e-04],
         [-4.267797e-04, -4.378001e-04],
         [-8.308084e-04, -6.774946e-04],
         ...,
         [-1.668270e-04, -1.206314e-03],
         [-2.169051e-04, -1.047814e-03],
         [-2.559629e-04, -7.607962e-04]],
 
        [[-5.571990e-06, -7.557722e-04],
         [ 3.563199e-04, -6.197994e-04],
         [ 7.374978e-04, -3.760532e-04],
         ...,
         [ 5.179913e-05,  7.124698e-05],
         [ 6.606088e-05